<a href="https://colab.research.google.com/github/WishmaPathirage/RIskLensUI/blob/main/mlmodel/Copy_of_UpdateRisk(train_using_1_ds_and_test_using_other).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Section A: Environment**

In [ ]:
# @title 1 - A1
#Cell 1

# 1. Remove the conflicting packages
!pip uninstall -y torch torchvision torchaudio fastai

# 2. Install the specific stable versions that work together
!pip install -q torch==2.5.1 transformers==4.46.3 datasets evaluate scikit-learn accelerate seaborn

import os
# Restart to finalize
os.kill(os.getpid(), 9)

Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Found existing installation: fastai 2.8.6
Uninstalling fastai-2.8.6:
  Successfully uninstalled fastai-2.8.6
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 162.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 107.7 MB/s eta 0:00:00
   

# **Section B: Train RiskLens model on SALT-NLP (with group split)**

In [ ]:
# @title 3 - B1
from datasets import load_dataset, Dataset
import random

ds_stream = load_dataset("SALT-NLP/search_privacy_risk", split="test", streaming=True)

rows = []
MAX_DIALOGS = 3000
count_dialogs = 0

for ex in ds_stream:
    group_id = f"{ex['example_id']}_{ex['log_id']}"   # keep one convo together
    history = ex["eval"]["history"]

    for h in history:
        details = h.get("details") or {}
        body = details.get("body")

        evaluation = h.get("evaluation") or {}
        label = evaluation.get("label")

        if not body:
            continue

        y = 1 if label == "LEAK" else 0
        rows.append({"text": body, "label": y, "group_id": group_id})

    count_dialogs += 1
    if count_dialogs >= MAX_DIALOGS:
        break

ds_all = Dataset.from_list(rows).shuffle(seed=42)
print(ds_all)
print("Leak count:", sum(ds_all["label"]), "Total:", len(ds_all))


Resolving data files:   0%|          | 0/219 [00:00<?, ?it/s]

Dataset({
    features: ['text', 'label', 'group_id'],
    num_rows: 12031
})
Leak count: 427 Total: 12031


In [ ]:
# @title 4 - B2
groups = list(set(ds_all["group_id"]))
random.seed(42)
random.shuffle(groups)

n = len(groups)
train_groups = set(groups[: int(0.8 * n)])
val_groups   = set(groups[int(0.8 * n): int(0.9 * n)])
test_groups  = set(groups[int(0.9 * n):])

def in_groups(example, group_set):
    return example["group_id"] in group_set

ds_train = ds_all.filter(lambda x: in_groups(x, train_groups))
ds_val   = ds_all.filter(lambda x: in_groups(x, val_groups))
ds_test  = ds_all.filter(lambda x: in_groups(x, test_groups))

print("Train:", len(ds_train), "Val:", len(ds_val), "Test:", len(ds_test))
print("Train leak:", sum(ds_train["label"]), "Val leak:", sum(ds_val["label"]), "Test leak:", sum(ds_test["label"]))


Filter:   0%|          | 0/12031 [00:00<?, ? examples/s]

Filter:   0%|          | 0/12031 [00:00<?, ? examples/s]

Filter:   0%|          | 0/12031 [00:00<?, ? examples/s]

Train: 9548 Val: 1108 Test: 1375
Train leak: 333 Val leak: 36 Test leak: 58


In [ ]:
# @title 5 - B3
import numpy as np
import torch
import evaluate
from collections import Counter
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tok(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)

train_tok = ds_train.map(tok, batched=True)
val_tok   = ds_val.map(tok, batched=True)
test_tok  = ds_test.map(tok, batched=True)


train_tok = train_tok.rename_column("label", "labels")
val_tok   = val_tok.rename_column("label", "labels")
test_tok  = test_tok.rename_column("label", "labels")

cols = ["input_ids", "attention_mask", "labels"]
train_tok.set_format(type="torch", columns=cols)
val_tok.set_format(type="torch", columns=cols)
test_tok.set_format(type="torch", columns=cols)

# class weights from TRAIN only
y_train = np.array(ds_train["label"])
counts = Counter(y_train)
neg, pos = counts.get(0, 0), counts.get(1, 0)
w1 = (neg / pos) if pos > 0 else 1.0
class_weights = torch.tensor([1.0, w1], dtype=torch.float)

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        weights = class_weights.to(logits.device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=weights)
        loss = loss_fct(logits.view(-1, model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

metric_f1 = evaluate.load("f1")
metric_acc = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": metric_acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1": metric_f1.compute(predictions=preds, references=labels)["f1"],
    }

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

args = TrainingArguments(
    output_dir="risklens_ft",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="none",
)

trainer = WeightedTrainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()
print("Best validation:", trainer.evaluate())


Map:   0%|          | 0/9548 [00:00<?, ? examples/s]

Map:   0%|          | 0/1108 [00:00<?, ? examples/s]

Map:   0%|          | 0/1375 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.653900,0.261744,0.984657,0.767123
2,0.243700,0.360959,0.987365,0.800000
3,0.159400,0.408860,0.988267,0.816901
4,0.060200,0.433999,0.986462,0.794521
5,0.048400,0.479248,0.985560,0.777778


Best validation: {'eval_loss': 0.4088597297668457, 'eval_accuracy': 0.9882671480144405, 'eval_f1': 0.8169014084507042, 'eval_runtime': 1.9013, 'eval_samples_per_second': 582.761, 'eval_steps_per_second': 36.817, 'epoch': 5.0}


In [ ]:
# @title 6 - B4
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve
import torch

pred = trainer.predict(test_tok)
logits = pred.predictions
y_true = pred.label_ids

probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]

precision, recall, thresholds = precision_recall_curve(y_true, probs)
f1_scores = (2 * precision * recall) / (precision + recall + 1e-12)

best_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_idx] if best_idx < len(thresholds) else 0.5

y_pred = (probs >= best_threshold).astype(int)

print("Best threshold:", best_threshold)
print("Best F1:", f1_scores[best_idx])
print("Precision:", precision[best_idx], "Recall:", recall[best_idx])
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("\nReport:\n", classification_report(y_true, y_pred, digits=4))
trainer.save_model("risklens_pretrained")
tokenizer.save_pretrained("risklens_pretrained")



Best threshold: 0.9848813
Best F1: 0.9166666666661673
Precision: 0.8870967741935484 Recall: 0.9482758620689655
Confusion Matrix:
 [[1310    7]
 [   3   55]]

Report:
               precision    recall  f1-score   support

           0     0.9977    0.9947    0.9962      1317
           1     0.8871    0.9483    0.9167        58

    accuracy                         0.9927      1375
   macro avg     0.9424    0.9715    0.9564      1375
weighted avg     0.9930    0.9927    0.9928      1375



('risklens_pretrained/tokenizer_config.json',
 'risklens_pretrained/special_tokens_map.json',
 'risklens_pretrained/vocab.txt',
 'risklens_pretrained/added_tokens.json',
 'risklens_pretrained/tokenizer.json')

# **Section C: Build Enron “cross-domain” dataset (privacy-labeled)**

In [ ]:
# @title 8 - C1
from datasets import load_dataset

enron = load_dataset("SetFit/enron_spam")
print(enron)
print(enron["train"][0].keys())
print(enron["train"][0])


README.md:   0%|          | 0.00/176 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/101M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/6.27M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/31716 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['message_id', 'text', 'label', 'label_text', 'subject', 'message', 'date'],
        num_rows: 31716
    })
    test: Dataset({
        features: ['message_id', 'text', 'label', 'label_text', 'subject', 'message', 'date'],
        num_rows: 2000
    })
})
dict_keys(['message_id', 'text', 'label', 'label_text', 'subject', 'message', 'date'])
{'message_id': 33214, 'text': 'any software just for 15 $ - 99 $ understanding oem software\nlead me not into temptation ; i can find the way myself .\n# 3533 . the law disregards trifles .', 'label': 1, 'label_text': 'spam', 'subject': 'any software just for 15 $ - 99 $', 'message': 'understanding oem software\nlead me not into temptation ; i can find the way myself .\n# 3533 . the law disregards trifles .', 'date': datetime.datetime(2005, 6, 18, 0, 0)}


In [ ]:
# @title 9 - C2
import re
from datasets import DatasetDict

PHONE_RE = re.compile(r"(\+?\d{1,3}[\s-]?)?(\(?\d{2,4}\)?[\s-]?)?\d{3,4}[\s-]?\d{4}")
EMAIL_RE = re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}")
CARD_RE  = re.compile(r"\b(?:\d[ -]*?){13,19}\b")  # rough card-like
BANK_RE  = re.compile(r"\b(account|acct|iban|swift|routing)\b", re.IGNORECASE)
ADDR_RE  = re.compile(r"\b\d{1,5}\s+\w+(?:\s+\w+){0,4}\s+(street|st|road|rd|ave|avenue|lane|ln|blvd|drive|dr)\b", re.IGNORECASE)
ID_RE    = re.compile(r"\b(ssn|nic|passport|national id)\b", re.IGNORECASE)

def is_privacy_leak(text: str) -> int:
    if text is None:
        return 0
    t = text.strip()
    if len(t) < 10:
        return 0
    hits = 0
    hits += 1 if PHONE_RE.search(t) else 0
    hits += 1 if EMAIL_RE.search(t) else 0
    hits += 1 if CARD_RE.search(t) else 0
    hits += 1 if BANK_RE.search(t) else 0
    hits += 1 if ADDR_RE.search(t) else 0
    hits += 1 if ID_RE.search(t) else 0
    return 1 if hits >= 1 else 0  # start simple: any hit means leak

def make_privacy_labels(example):
    txt = example.get("text") or example.get("message") or example.get("email") or ""
    return {"text": txt, "label": is_privacy_leak(txt)}

# Apply on both train and test splits if available
splits = {}
for split in enron.keys():
    splits[split] = enron[split].map(make_privacy_labels)

enron_priv = DatasetDict(splits)
print(enron_priv)
print(enron_priv["train"][0])


Map:   0%|          | 0/31716 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['message_id', 'text', 'label', 'label_text', 'subject', 'message', 'date'],
        num_rows: 31716
    })
    test: Dataset({
        features: ['message_id', 'text', 'label', 'label_text', 'subject', 'message', 'date'],
        num_rows: 2000
    })
})
{'message_id': 33214, 'text': 'any software just for 15 $ - 99 $ understanding oem software\nlead me not into temptation ; i can find the way myself .\n# 3533 . the law disregards trifles .', 'label': 0, 'label_text': 'spam', 'subject': 'any software just for 15 $ - 99 $', 'message': 'understanding oem software\nlead me not into temptation ; i can find the way myself .\n# 3533 . the law disregards trifles .', 'date': datetime.datetime(2005, 6, 18, 0, 0)}


In [ ]:
# @title 10 - C3
from collections import Counter
print("Train label counts:", Counter(enron_priv["train"]["label"]))
if "test" in enron_priv:
    print("Test label counts:", Counter(enron_priv["test"]["label"]))


Train label counts: Counter({0: 26014, 1: 5702})
Test label counts: Counter({0: 1680, 1: 320})


In [ ]:
# @title 11 - C4
from datasets import DatasetDict

# Make validation split from train
tmp = enron_priv["train"].train_test_split(test_size=0.2, seed=42)
train_ds = tmp["train"]
val_ds   = tmp["test"]

# Use existing test if present, otherwise split from remaining
if "test" in enron_priv:
    test_ds = enron_priv["test"]
else:
    test_ds = val_ds

ds = DatasetDict({"train": train_ds, "validation": val_ds, "test": test_ds})
print(ds)


DatasetDict({
    train: Dataset({
        features: ['message_id', 'text', 'label', 'label_text', 'subject', 'message', 'date'],
        num_rows: 25372
    })
    validation: Dataset({
        features: ['message_id', 'text', 'label', 'label_text', 'subject', 'message', 'date'],
        num_rows: 6344
    })
    test: Dataset({
        features: ['message_id', 'text', 'label', 'label_text', 'subject', 'message', 'date'],
        num_rows: 2000
    })
})


In [ ]:
# @title D1 - Load SALT-trained model for cross-domain testing
from transformers import AutoTokenizer, AutoModelForSequenceClassification

BASE_MODEL_DIR = "risklens_pretrained"  # from Cell B4 save

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_DIR, local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL_DIR, local_files_only=True)

print("Loaded SALT-trained model:", BASE_MODEL_DIR)


In [ ]:
# @title D2 - Cross-domain evaluation on Enron (NO training)
import numpy as np
import torch
from sklearn.metrics import precision_recall_curve, confusion_matrix, classification_report

def tok(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)

enron_tok = ds.map(tok, batched=True)
enron_tok = enron_tok.rename_column("label", "labels")
enron_tok.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# --- Predict on validation to find best threshold ---
val_pred = Trainer(model=model).predict(enron_tok["validation"])
val_probs = torch.softmax(torch.tensor(val_pred.predictions), dim=1).numpy()[:, 1]
y_val = val_pred.label_ids

precision, recall, thresholds = precision_recall_curve(y_val, val_probs)
f1_scores = (2 * precision * recall) / (precision + recall + 1e-12)

best_idx = int(np.argmax(f1_scores))
best_threshold = float(thresholds[best_idx]) if best_idx < len(thresholds) else 0.5

print("Cross-domain best threshold (val):", best_threshold)
print("Cross-domain best F1 (val):", float(f1_scores[best_idx]))

# --- Final test evaluation using threshold ---
test_pred = Trainer(model=model).predict(enron_tok["test"])
test_probs = torch.softmax(torch.tensor(test_pred.predictions), dim=1).numpy()[:, 1]
y_test = test_pred.label_ids

y_hat = (test_probs >= best_threshold).astype(int)

print("\nCross-domain Confusion Matrix:\n", confusion_matrix(y_test, y_hat))
print("\nCross-domain Classification Report:\n")
print(classification_report(y_test, y_hat, digits=4))


In [ ]:
# @title 12
from transformers import AutoTokenizer, AutoModelForSequenceClassification

BASE_MODEL_DIR = "risklens_pretrained"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_DIR, local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL_DIR, local_files_only=True)

print("Loaded:", BASE_MODEL_DIR)
print("Labels:", model.config.id2label)


Loaded: risklens_pretrained
Labels: {0: 'LABEL_0', 1: 'LABEL_1'}


In [ ]:
# @title 13
def tok(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)

tokenized = ds.map(tok, batched=True)
tokenized = tokenized.rename_column("label", "labels")
tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
print(tokenized)
print(tokenized["train"][0])


Map:   0%|          | 0/25372 [00:00<?, ? examples/s]

Map:   0%|          | 0/6344 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['message_id', 'text', 'labels', 'label_text', 'subject', 'message', 'date', 'input_ids', 'attention_mask'],
        num_rows: 25372
    })
    validation: Dataset({
        features: ['message_id', 'text', 'labels', 'label_text', 'subject', 'message', 'date', 'input_ids', 'attention_mask'],
        num_rows: 6344
    })
    test: Dataset({
        features: ['message_id', 'text', 'labels', 'label_text', 'subject', 'message', 'date', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})
{'labels': tensor(0), 'input_ids': tensor([  101,  3237,  8013,  2862,  2065,  2057,  2064,  4390,  2017,  2320,
         2153,  1010,  3889,  3791,  1010,  2011,  1996,  2203,  1997,  2651,
         1010,  1996,  3415,  1998,  3616,  1997,  1996,  3145, 12706,  2006,
         1996,  2862,  4987,  1012,  3531,  3143,  1996,  2171,  1997,  1996,
         2343,  1010,  3580,  2343,  1998,  3042,  3616,  1012,   102,     0,
            0,     

In [ ]:
# @title 14
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer

acc = evaluate.load("accuracy")
f1  = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels, average="binary")["f1"],
    }

args = TrainingArguments(
    output_dir="risklens_enron_ft",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.104500,0.212475,0.946564,0.847091
2,0.095800,0.159613,0.961854,0.882410
3,0.052600,0.251908,0.959647,0.879925
4,0.019500,0.275806,0.963745,0.890995
5,0.006900,0.299433,0.959332,0.881434


{'eval_loss': 0.2994328439235687,
 'eval_accuracy': 0.9593316519546028,
 'eval_f1': 0.8814338235294118,
 'eval_runtime': 10.4607,
 'eval_samples_per_second': 606.46,
 'eval_steps_per_second': 37.952,
 'epoch': 5.0}

In [ ]:
# @title 15
import numpy as np
import torch
from sklearn.metrics import precision_recall_curve, confusion_matrix, classification_report

# 1) Get probs on validation
pred_val = trainer.predict(tokenized["validation"])
logits_val = pred_val.predictions
y_val = pred_val.label_ids

probs_val = torch.softmax(torch.tensor(logits_val), dim=1).numpy()[:, 1]

# 2) Find best threshold by F1
precision, recall, thresholds = precision_recall_curve(y_val, probs_val)
f1_scores = (2 * precision * recall) / (precision + recall + 1e-12)

best_idx = int(np.argmax(f1_scores))
best_threshold = float(thresholds[best_idx]) if best_idx < len(thresholds) else 0.5

print("Best threshold (by F1 on validation):", best_threshold)
print("Best F1:", float(f1_scores[best_idx]))
print("Precision:", float(precision[best_idx]), "Recall:", float(recall[best_idx]))


Best threshold (by F1 on validation): 0.9691686034202576
Best F1: 0.8853717026373907
Precision: 0.9313824419778002 Recall: 0.8436928702010968


In [ ]:
# @title 16
# Predict on test
pred_test = trainer.predict(tokenized["test"])
logits_test = pred_test.predictions
y_test = pred_test.label_ids

probs_test = torch.softmax(torch.tensor(logits_test), dim=1).numpy()[:, 1]
y_pred = (probs_test >= best_threshold).astype(int)

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred, digits=4))


Confusion Matrix:
 [[1664   16]
 [  55  265]]

Classification Report:

              precision    recall  f1-score   support

           0     0.9680    0.9905    0.9791      1680
           1     0.9431    0.8281    0.8819       320

    accuracy                         0.9645      2000
   macro avg     0.9555    0.9093    0.9305      2000
weighted avg     0.9640    0.9645    0.9636      2000



In [ ]:
# @title 17
import os, json
from datetime import datetime

SAVE_DIR = "risklens_enron_generalized"
os.makedirs(SAVE_DIR, exist_ok=True)

trainer.model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

meta = {
    "saved_at": datetime.utcnow().isoformat() + "Z",
    "base_model": "distilbert-base-uncased",
    "fine_tuned_on": "SetFit/enron_spam + rule-based privacy labels",
    "best_threshold_val_f1": best_threshold,
    "val_metrics_last": {
        "eval_accuracy": 0.955233291298865,
        "eval_f1": 0.8614634146341463
    },
    "labels": { "0": "NO_LEAK", "1": "LEAK" },
    "notes": "Threshold tuned on validation split using PR curve F1."
}

meta_path = os.path.join(SAVE_DIR, "risklens_meta.json")
with open(meta_path, "w") as f:
    json.dump(meta, f, indent=2)

print("Saved model + tokenizer to:", SAVE_DIR)
print("Saved meta to:", meta_path)


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Saved model + tokenizer to: risklens_enron_generalized
Saved meta to: risklens_enron_generalized/risklens_meta.json


/tmp/ipython-input-2228755248.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "saved_at": datetime.utcnow().isoformat() + "Z",


# **Part B: NER Module using WNUT-2017**

In [ ]:
!pip -q uninstall -y datasets huggingface_hub transformers
!pip -q install -U datasets huggingface_hub transformers accelerate evaluate seqeval


In [ ]:
from datasets import load_dataset
wnut = load_dataset("tner/wnut2017", trust_remote_code=True)
wnut


`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'tner/wnut2017' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'tner/wnut2017' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this s

RuntimeError: Dataset scripts are no longer supported, but found wnut2017.py

In [ ]:
# tner/wnut2017 usually provides a ClassLabel feature
features = wnut["train"].features
tag_feature = None

# try common keys
for k in ["ner_tags", "tags"]:
    if k in features:
        tag_feature = features[k]
        tag_key = k
        break

print("Tag key:", tag_key)
label_list = tag_feature.feature.names if hasattr(tag_feature, "feature") else tag_feature.names
id2label = {i: l for i, l in enumerate(label_list)}
label2id = {l: i for i, l in enumerate(label_list)}

print("Num labels:", len(label_list))
print(label_list[:10])


NameError: name 'wnut' is not defined